# qSonify Hello World: Hearing Algorithms to Aid Algorithmic Abstraction
#### https://github.com/jiosue/qSonify

We will show how to use qSonify to recognize patterns in quantum algorithms in order to help us understand their decomposition and/or abstraction.

In [16]:
import qSonify

For our example, the algorithms are shown below.

In [18]:
alg1 = ["h(0)", "cx(0, 1)", "cx(0, 2)", "cx(0, 3)", "cx(0, 4)"]
alg2 = ['h(0)', 'rz(pi, 2)', 'cx(2, 3)', 'rx(pi/2, 2)', 'cx(0, 3)']

Next we have to choose a mapping from output to midi file. Many mappings require us to choose a set of notes to use, let's define ours here.

In [19]:
notes = ("c4", "d4", "e4", "f4", "g4", "a4", "b4", "c5")

To run our algorithms, we'll use IBM's simulator. If you want to run on actual hardware, note:

- You must have your IBM account stored locally. You only need to do this once, via `qiskit.IBMQ.store_account(APItoken)`
- If you run your algorithm with markovian sampling, you will sit in the queue many times, because you need to wait for the previous output in order to run the next circuit (see the discussion in the cells below)

In [21]:
backend = qSonify.simulator
# To run on the actual quantum computer, uncomment one of the following lines.
# backend = "ibmqx4"
# backend = "ibmqx5"

Now let's define/choose our mapping. We can either define our own mapping or choose one of the one's already implemented.

In [22]:
#mapping = qSonify.fermionic(notes=notes)
#mapping = qSonify.scale(notes=notes)
#mapping = qSonify.grandpiano(low_notes=("c3", "d3", "e3"), high_notes=notes)
#mapping = qSonify.stringquartet()
#mapping = qSonify.frequencymapping(low_freq=300, base=4)

# user_defined mapping must be of this form.
# as an examle, this particular user fefined mapping is
# the same as qSonify.frequencymapping(low_freq=300, base=4).
# To see more details of how to add more tracks, etc, see the
# `maps` folder.
def mapping(res, name, tempo):
    """
    res: list of strings, outputs of the quantum computer run, ie
         ['10010', '01101', ...]
    name: str, name of the song.
    tempo: int, tempo of the song.
    return: qSonify.Song object.
    """
    s = qSonify.Song(name=name, tempo=tempo, num_tracks=1)
    for x in res:
        note = qSonify.freq_to_note(int(x, base=4) + 300)
        s.addNote(note, duration=.5) # eigth notes
    return s

Now let's run it! We use the function with the following keyword arguments:

    qSonify.alg_to_song(algorithm, num_qubits, num_samples, backend, markovian, mapping, name, tempo):
        algorithm: algorithm (list of strings), NOT a list of algorithms, 
                   each string is a gate in GATE_ARGUMENTS.keys() with whatever 
                   arguments required to define the gate.
        num_qubits: int, number of qubits to run each algorithm on. Not required.
        num_samples: int, number of samples to take from the quantum computer,
                          for most mappings this is equal to the number of beats.
        backend: str, IBM backend to run the algorithm on.
        markovian: bool, whether to sample using qc.sample or qc.markovian_sample.
        mapping: function, which mapping from output to sound to use.
        name: str, name of song.
        tempo: int, tempo of song.
        returns: Song object

Markovian sampling is inputting the output of the previous run in as the input to the next run. Note that when running on actual IBM hardware, this will make you sit in the queue `num_samples` times! However, using markovian sampling usually yields better results for hearing the algorithm.

In [23]:
kwargs = dict(num_samples=40, backend=backend, markovian=True, mapping=mapping, tempo=150)

s1 = qSonify.alg_to_song(alg1, name="HelloWorld_alg1", **kwargs)
s2 = qSonify.alg_to_song(alg2, name="HelloWorld_alg2", **kwargs)

Now let's listen to them! *Note the play function only works on Windows right now, because it just calls the system to open the midi file, which Windows by default opens in Windows Media Player. To play the files on your computer if it is non Windows, go to the folder output/ and open the saved .mid file however you want.*

In [33]:
s1.play()

In [34]:
s2.play()

Now let's see if we can still "hear" these algorithms when they are embedded within another circuit.

In [26]:
alg1_emb = ["h(4)", "rz(pi/8, 2)"] + alg1 + ["h(1)"]
alg2_emb = ["h(2)", "rz(pi/4, 0)"] + alg2 + ["h(0)"]
s1_emb = qSonify.alg_to_song(alg1_emb, name="HelloWorld_alg1_emb", **kwargs)
s2_emb = qSonify.alg_to_song(alg2_emb, name="HelloWorld_alg2_emb", **kwargs)

In [27]:
s1_emb.play() # compare to s1.play()

In [29]:
s2_emb.play() # compare to s2.play()

Can you hear the similarities? Do you hear the original algorithm embedded within the new one? Now let's try one more thing. Let see if we can hear both algorithm1 and algorithm2 when they are performed in the same algorithm and embedded within other gate sequences.

In [31]:
alg12_emb = alg1_emb + alg2_emb + alg2_emb + alg1_emb
s12_emb = qSonify.alg_to_song(alg12_emb, name="HelloWorld_alg12_emb", **kwargs)

Now let's listen to it. Try and compare `s12_emb` to `s1`, `s2`, `s1_emb`, and `s2_emb`, and try to see if you can hear the original `alg1` and `alg2`.

In [35]:
s12_emb.play()

So did you hear it? Sonifying quantum algorithms in this way could help researchers "decompile" algorithms that are represented in terms of elementary gate operations that are generally hard to abstract from. Try testing some different mappings or defining your own! All the midi file outputs from `s.play()` or `s.view()` or `s.writeFile()` are saved in the "output" folder in this same directory. By the way, the `s.view()` functionality is computer specific, and will probably not work on your machine. I call MuseScore on my computer to open the midi file. Again, on your machine, go to the "output/" folder and open the midi file in any way that you want.

In [ ]:
s1.view()
s2.view()
s1_emb.view()
s2_emb.view()
s12_emb.view()

Do you think it is easier to "see" the embedded algorithms or "hear" them? Sonification is a powerful tool because our ears can often hear patterns that are difficult for our eyes to pick out.